### ORBIT Example - ParametricManager

ParametricManager provides a similar interface into ORBIT as ProjectManager but allows for some (or all) inputs to be parameterized. This class is useful for quickly exploring how a module or project scales with certain inputs.

In [1]:
from ORBIT.phases.design import MonopileDesign

from ORBIT import ParametricManager


### Monopile Design Example
Perform a parametric sweep of site depth and mean wind speed to see the effects on monopile capex.

In [2]:
# For this example we will look at the MonopileDesign module.
MonopileDesign.expected_config

{'site': {'depth': 'm', 'mean_windspeed': 'm/s'},
 'plant': {'num_turbines': 'int'},
 'turbine': {'rotor_diameter': 'm',
  'hub_height': 'm',
  'rated_windspeed': 'm/s'},
 'monopile_design': {'yield_stress': 'Pa (optional)',
  'load_factor': 'float (optional)',
  'material_factor': 'float (optional)',
  'monopile_density': 'kg/m3 (optional)',
  'monopile_modulus': 'Pa (optional)',
  'monopile_tp_connection_thickness': 'm (optional)',
  'transition_piece_density': 'kg/m3 (optional)',
  'transition_piece_thickness': 'm (optional)',
  'transition_piece_length': 'm (optional)',
  'soil_coefficient': 'N/m3 (optional)',
  'air_density': 'kg/m3 (optional)',
  'weibull_scale_factor': 'float (optional)',
  'weibull_shape_factor': 'float (optional)',
  'turb_length_scale': 'm (optional)',
  'monopile_steel_cost': 'USD/t (optional)',
  'tp_steel_cost': 'USD/t (optional)'}}

In [3]:
# The following inputs are the only "required" inputs for the MonopileDesign module.
# Thee 'site' inputs are commented out as they will be defined as parametric inputs below.

base_config = {
#     "site": {
#         "depth": 20,
#         "mean_windspeed": 8
#     }
    "turbine": "12MW_generic",
    "plant": {
        "num_turbines": 50
    }
}

In [4]:
# Parametric inputs:

parameters = {
    "site.depth": [20, 40, 60],           # The dot-notation allows you to access nested dictionaries
    "site.mean_windspeed": [8, 9, 10]     # These inputs correspond to the 'depth' and 'mean_windspeed' inputs above, which are nested
}                                         # in the 'site' dictionary.

In [5]:
# Desired results:
# Since there are so many available results in ORBIT, you have to tell ParametricManager which ones
# you are interested in for this parametric run. The syntax for this always follows the
# 'lambda run: run.{output}' format. The {output} can be any output available for the configured module

results = {
    "capex": lambda run: run.total_cost
}

In [6]:
# Pass the above definitions into ParametricManager like this:
# Note: I only want to run MonopileDesign so I passed it into module directily.
# If you want to run an entire project (with multiple modules), leave module blank and it will
# automatically run ProjectManager

parametric = ParametricManager(base_config, parameters, results, module=MonopileDesign)
parametric.run()


# By default, ParametricManager doesn't run the product of all parameters and just
# runs the first set of inputs, then the second set of inputs, etc.
# In this case, there will only be 3 results
parametric.results

ORBIT library intialized at '/Users/nriccobo/GitHub/ORBIT/library'


,site.depth,site.mean_windspeed,capex
0,20,8,1.810642e+08
1,40,9,2.616899e+08
2,60,10,3.580034e+08


In [7]:
# To configure ParametricManager to run the product of all inputs, pass 'product=True'
# This will result in 3x3 results as each combination is ran
parametric = ParametricManager(base_config, parameters, results, module=MonopileDesign, product=True)
parametric.run()

parametric.results

,site.depth,site.mean_windspeed,capex
0,20,8,1.810642e+08
1,20,9,1.930323e+08
2,20,10,2.052779e+08
3,40,8,2.457850e+08
4,40,9,2.616899e+08
5,40,10,2.779403e+08
6,60,8,3.171299e+08
7,60,9,3.373573e+08
8,60,10,3.580034e+08


In [8]:
# If you are configuring many parameters it can take a long time to run, especially if weather is turned on.
# To get an idea of how long it'll take, you can run the .preview() method:

parameters = {
    "site.depth": [20, 40, 60],
    "site.mean_windspeed": [8, 9, 10],
    "monopile_design.soil_coefficient": [4000000, 4500000, 5000000]
}

parametric = ParametricManager(base_config, parameters, results, module=MonopileDesign, product=True)
parametric.preview()

10 runs elapsed time: 0.03s
27 runs estimated time: 0.07s


,site.depth,site.mean_windspeed,monopile_design.soil_coefficient,capex
0,40,10,4000000,2.779403e+08
1,40,10,5000000,2.744995e+08
2,40,9,4500000,2.599891e+08
3,40,8,4500000,2.442139e+08
4,60,9,4000000,3.373573e+08
5,40,9,4000000,2.616899e+08
6,40,8,5000000,2.428396e+08
7,60,10,5000000,3.541935e+08
8,20,9,4500000,1.915154e+08
9,20,10,4500000,2.036411e+08
